# DS-SF-27 | Unit Project 3: Basic Machine Learning Modeling

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [16]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.notebook_repr_html', True)

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import linear_model

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import seaborn as sns

In [17]:
df = pd.read_csv(os.path.join('..', '..', 'dataset', 'ucla-admissions.csv'))
df.dropna(inplace = True)

df

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
...,...,...,...,...
395,0,620.0,4.00,2.0
396,0,560.0,3.04,3.0
397,0,460.0,2.63,2.0
398,0,700.0,3.65,2.0


In [18]:
df.groupby('prestige').mean()

,admit,gre,gpa
prestige,,,
1.0,0.540984,611.803279,3.453115
2.0,0.358108,596.621622,3.367365
3.0,0.231405,574.876033,3.432893
4.0,0.179104,570.149254,3.318358


In [19]:
df.duplicated().sum()

5

## Part A.  Frequency Table

> ### Question 1.  Create a frequency table for `prestige` and whether or not an applicant was admitted.

In [20]:
df.prestige.value_counts(dropna = False).sort_index()

1.0     61
2.0    148
3.0    121
4.0     67
Name: prestige, dtype: int64

In [21]:
pd.crosstab(df.prestige, df.admit,
           rownames = ['Prestige'], 
           colnames = ['Admit'])


Admit,0,1
Prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part B.  Variable Transformations

In [22]:
#Question 2. Create a one-hot encoding for prestige.

In [23]:
# TODO
prestige_df = pd.get_dummies(df.prestige, prefix = 'prestige')

In [24]:
prestige_df

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
...,...,...,...,...
395,0.0,1.0,0.0,0.0
396,0.0,0.0,1.0,0.0
397,0.0,1.0,0.0,0.0
398,0.0,1.0,0.0,0.0


In [25]:
prestige_df.rename(columns = {'prestige_1.0': 'prestige_1',
                           'prestige_2.0': 'prestige_2',
                           'prestige_3.0': 'prestige_3',
                           'prestige_4.0': 'prestige_4'}, inplace = True)

In [26]:
prestige_df

,prestige_1,prestige_2,prestige_3,prestige_4
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
...,...,...,...,...
395,0.0,1.0,0.0,0.0
396,0.0,0.0,1.0,0.0
397,0.0,1.0,0.0,0.0
398,0.0,1.0,0.0,0.0


> ### Question 3.  How many of these binary variables do we need for modeling?

Answer: three...knowing three values we will know the forth one. 

> ### Question 4.  Why are we doing this?

Answer: We are transforming Prestige feature with 4 possible values into binary features, with only one active.

> ### Question 5.  Add all these binary variables in the dataset and remove the now redundant `prestige` feature.

In [27]:
df = df.join([prestige_df])

In [28]:
df.columns

Index([u'admit', u'gre', u'gpa', u'prestige', u'prestige_1', u'prestige_2',
       u'prestige_3', u'prestige_4'],
      dtype='object')

In [29]:
df.drop('prestige', axis = 1, inplace = True)

In [30]:
df

,admit,gre,gpa,prestige_1,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,0.0,0.0,1.0,0.0
1,1,660.0,3.67,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
395,0,620.0,4.00,0.0,1.0,0.0,0.0
396,0,560.0,3.04,0.0,0.0,1.0,0.0
397,0,460.0,2.63,0.0,1.0,0.0,0.0
398,0,700.0,3.65,0.0,1.0,0.0,0.0


In [17]:
#df.drop('prestige_1', axis = 1, inplace = True)

In [18]:
#df

In [19]:
df.columns

Index([u'admit', u'gre', u'gpa', u'prestige_1', u'prestige_2', u'prestige_3',
       u'prestige_4'],
      dtype='object')

## Part C.  Hand calculating odds ratios

Let's develop our intuition about expected outcomes by hand calculating odds ratios.

> ### Question 6.  Create a frequency table for `prestige = 1` and whether or not an applicant was admitted.

In [20]:
# TODO
##df[df.prestige_1]
#pd.crosstab(df.prestige_1, df.admit, dropna = False)
pd.crosstab(df.prestige_1, df.admit,
           rownames = ['Prestige'], 
           colnames = ['Admit'])

Admit,0,1
Prestige,,
0.0,243,93
1.0,28,33


In [21]:
#TODO
#adm = df[df.prestige_1]
#pd.crosstab(adm.admit, adm.prestige, dropna = False)

> ### Question 7.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the most prestigious undergraduate schools.

In [22]:
#TODO
prob_1 = 33 / 61.
odds_1 = prob_1 / (1 - prob_1)

print prob_1
print odds_1

0.540983606557
1.17857142857


In [ ]:
#odds of admissin is 1.18 more than one.

> ### Question 8.  Now calculate the odds of admission for undergraduates who did not attend a #1 ranked college.

In [23]:
# TODO
prob_2 = 93 / 336.
odds_2 = prob_2 / (1 - prob_2)

print prob_2
print odds_2

0.276785714286
0.382716049383


In [ ]:
#odds are less than one -its against it

> ### Question 9.  Finally, what's the odds ratio?

In [24]:
# TODO
odds_1 / odds_2

3.079493087557604

> ### Question 10.  Write this finding in a sentenance.

Answer: From the frequency table for prestige = 1, a total of 33 applicants will be admited while 28 won't be admited to graduate school.  In terms of probability, there is a 54% of chance that an applicant will be admited and an odd of 1.18. In the other hand, the probablity of not being admited is 28% and an odd of 0.38. The odds ratio is 3, meaning that an applicant is three times more likely to be admited to graduate school if he/she is coming from a prestigeous undergraduate school. 

> ### Question 11.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the least prestigious undergraduate schools.  Then calculate their odds ratio of being admitted to UCLA.  Finally, write this finding in a sentenance.

In [25]:
# TODO
pd.crosstab(df.prestige_4, df.admit,
           rownames = ['Prestige'], 
           colnames = ['Admit'])

Admit,0,1
Prestige,,
0.0,216,114
1.0,55,12


In [26]:
prob_3 = 12 / 67.
odds_3 = prob_3 / (1 - prob_3)

print prob_3
print odds_3

0.179104477612
0.218181818182


Answer: If the applicant attend the least prestigeous undergraduate school, the probability of getting accepted to graduate school is 18% and an odd of 0.2.

## Part C. Analysis using `statsmodels`

> ### Question 12.  Fit a logistic regression model prediting admission into UCLA using `gre`, `gpa`, and the prestige of the undergraduate schools.  Use the highest prestige undergraduate schools as your reference point.

In [30]:
model = smf.logit(formula = 'admit ~ gre + gpa + prestige_2 + prestige_3 + prestige_4', data = df).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Tue, 25 Oct 2016   Pseudo R-squ.:                 0.08166
Time:                        11:54:44   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -3.8769      1.142     -3.393      0.001        -6.116    -1.638
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
==============================================================================
"""

In [ ]:
#they are all significant

> ### Question 13.  Print the model's summary results.

In [31]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Tue, 25 Oct 2016   Pseudo R-squ.:                 0.08166
Time:                        11:55:17   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -3.8769      1.142     -3.393      0.001        -6.116    -1.638
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
==============================================================================
"""

> ### Question 14.  What are the odds ratios of the different features and their 95% confidence intervals?

In [9]:
# TODO
#logit= log P/1-P
#log odds
#check class 9
np.exp(model)

NameError: name 'model' is not defined

In [ ]:
np.exp(model.conf_int(alpha = 0.5)).\
rename (columns)

> ### Question 15.  Interpret the odds ratio for `prestige = 2`.

Answer: i don't know how to interprete negative odds.

> ### Question 16.  Interpret the odds ratio of `gpa`.

Answer:

> ### Question 17.  Assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [ ]:
#predict X = pd.DataFrame [{'intercept' : [1,1,1,1]},
                         ]

In [41]:
# TODO
model.predict(pd.DataFrame([[800, 4, 1, 0, 0, 0]], columns=df.columns[1:]))

array([ 0.73403998])

In [45]:
model.predict(pd.DataFrame([[800, 4, 0, 1, 0, 0]], columns=df.columns[1:]))

array([ 0.58299512])

In [46]:
model.predict(pd.DataFrame([[800, 4, 0, 0, 1, 0]], columns=df.columns[1:]))

array([ 0.41983282])

In [47]:
model.predict(pd.DataFrame([[800, 4, 0, 0, 0,1]], columns=df.columns[1:]))

array([ 0.36860803])

Answer:the probability of admission from a tier-1 is 73%, tier-2 is 58%, tier-3 is 42% and tier-4 is 37%

## Part D. Moving the model from `statsmodels` to `sklearn`

> ### Question 18.  Let's assume we are satisfied with our model.  Remodel it (same features) using `sklearn`.  When creating the logistic regression model with `LogisticRegression(C = 10 ** 2)`.

In [11]:
# TODO
from sklearn.linear_model import LogisticRegression

In [32]:
#create instance of LogisticRegression
logreg = LogisticRegression(C=10 **2)
#logreg.fit(inputfeatures,outputfeature)
logreg.fit(df.drop(['admit','prestige_1'] ,axis=1, inplace=False),df['admit'])

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [33]:
logreg.predict_proba([[800, 4,0, 0, 0]])

array([[ 0.28814605,  0.71185395]])

> ### Question 19.  What are the odds ratios for the different variables and how do they compare with the odds ratios calculated with `statsmodels`?

In [ ]:
# TODO

Answer: 

> ### Question 20.  Again assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [ ]:
# TODO

Answer: See line 51, 52, 53, & 54. The probability of admission from a tier-1 is 73%, tier -2 is 57%, tier-3 is 41% and tier-4 is 36%